# Paraphrase

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/paraphrase](https://github.com/huseinzol05/Malaya/tree/master/example/paraphrase).
    
</div>

<div class="alert alert-warning">

This module only trained on standard language structure, so it is not save to use it for local language structure.
    
</div>

In [1]:
%%time

import malaya
from pprint import pprint

CPU times: user 4.83 s, sys: 714 ms, total: 5.54 s
Wall time: 4.52 s


### List available Transformer model

In [2]:
malaya.paraphrase.available_transformer()

INFO:root:tested on ParaSCI test set.


,Size (MB),Quantized Size (MB),BLEU
t5,1250.0,481.0,0.608904
small-t5,355.6,195.0,0.617456
tiny-t5,208.0,103.0,0.460321


### Load Transformer model

```python
def transformer(model: str = 'small-t5', quantized: bool = False, **kwargs):
    """
    Load Malaya transformer encoder-decoder model to generate a paraphrase given a string.

    Parameters
    ----------
    model : str, optional (default='small-t5')
        Model architecture supported. Allowed values:

        * ``'t5'`` - T5 BASE parameters.
        * ``'small-t5'`` - T5 SMALL parameters.
        * ``'tiny-t5'`` - T5 TINY parameters.

    quantized : bool, optional (default=False)
        if True, will load 8-bit quantized model.
        Quantized model not necessary faster, totally depends on the machine.

    Returns
    -------
    result: model
        List of model classes:

        * if `t5` in model, will return `malaya.model.t5.Paraphrase`.
    """
```

In [6]:
t5 = malaya.paraphrase.transformer(model = 'small-t5')

INFO:root:running paraphrase-v2/small-t5 using device /device:CPU:0


### Paraphrase simple string

We only provide `greedy_decoder` method for T5 models,

```python
def greedy_decoder(self, strings: List[str]):
    """
    paraphrase strings.

    Parameters
    ----------
    strings: List[str]

    Returns
    -------
    result: List[str]
    """
```

In [7]:
string = "Beliau yang juga saksi pendakwaan kesembilan berkata, ia bagi mengelak daripada wujud isu digunakan terhadap Najib."
pprint(string)

('Beliau yang juga saksi pendakwaan kesembilan berkata, ia bagi mengelak '
 'daripada wujud isu digunakan terhadap Najib.')


In [8]:
pprint(t5.greedy_decoder([string]))

['Beliau yang juga saksi pendakwaan kesembilan berkata, ia bagi mengelak wujud '
 'isu yang digunakan terhadap Najib.']


In [11]:
string = """
PELETAKAN jawatan Tun Dr Mahathir Mohamad sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) ditolak di dalam mesyuarat khas Majlis Pimpinan Tertinggi (MPT) pada 24 Februari lalu.

Justeru, tidak timbul soal peletakan jawatan itu sah atau tidak kerana ia sudah pun diputuskan pada peringkat parti yang dipersetujui semua termasuk Presiden, Tan Sri Muhyiddin Yassin.

Bekas Setiausaha Agung Bersatu Datuk Marzuki Yahya berkata, pada mesyuarat itu MPT sebulat suara menolak peletakan jawatan Dr Mahathir.

"Jadi ini agak berlawanan dengan keputusan yang kita sudah buat. Saya tak faham bagaimana Jabatan Pendaftar Pertubuhan Malaysia (JPPM) kata peletakan jawatan itu sah sedangkan kita sudah buat keputusan di dalam mesyuarat, bukan seorang dua yang buat keputusan.

"Semua keputusan mesti dibuat melalui parti. Walau apa juga perbincangan dibuat di luar daripada keputusan mesyuarat, ini bukan keputusan parti.

"Apa locus standy yang ada pada Setiausaha Kerja untuk membawa perkara ini kepada JPPM. Seharusnya ia dibawa kepada Setiausaha Agung sebagai pentadbir kepada parti," katanya kepada Harian Metro.

Beliau mengulas laporan media tempatan hari ini mengenai pengesahan JPPM bahawa Dr Mahathir tidak lagi menjadi Pengerusi Bersatu berikutan peletakan jawatannya di tengah-tengah pergolakan politik pada akhir Februari adalah sah.

Laporan itu juga menyatakan, kedudukan Muhyiddin Yassin memangku jawatan itu juga sah.

Menurutnya, memang betul Dr Mahathir menghantar surat peletakan jawatan, tetapi ditolak oleh MPT.

"Fasal yang disebut itu terpakai sekiranya berhenti atau diberhentikan, tetapi ini mesyuarat sudah menolak," katanya.

Marzuki turut mempersoal kenyataan media yang dibuat beberapa pimpinan parti itu hari ini yang menyatakan sokongan kepada Perikatan Nasional.

"Kenyataan media bukanlah keputusan rasmi. Walaupun kita buat 1,000 kenyataan sekali pun ia tetap tidak merubah keputusan yang sudah dibuat di dalam mesyuarat. Kita catat di dalam minit apa yang berlaku di dalam mesyuarat," katanya.
"""

In [13]:
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

string = cleaning(string)
splitted = malaya.text.function.split_into_sentences(string)
splitted

['PELETAKAN jawatan Tun Dr Mahathir Mohamad sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) ditolak di dalam mesyuarat khas Majlis Pimpinan Tertinggi (MPT) pada 24 Februari lalu.',
 'Justeru, tidak timbul soal peletakan jawatan itu sah atau tidak kerana ia sudah pun diputuskan pada peringkat parti yang dipersetujui semua termasuk Presiden, Tan Sri Muhyiddin Yassin.',
 'Bekas Setiausaha Agung Bersatu Datuk Marzuki Yahya berkata, pada mesyuarat itu MPT sebulat suara menolak peletakan jawatan Dr Mahathir.',
 '"Jadi ini agak berlawanan dengan keputusan yang kita sudah buat.',
 'Saya tak faham bagaimana Jabatan Pendaftar Pertubuhan Malaysia (JPPM) kata peletakan jawatan itu sah sedangkan kita sudah buat keputusan di dalam mesyuarat, bukan seorang dua yang buat keputusan.',
 '"Semua keputusan mesti dibuat melalui parti.',
 'Walau apa juga perbincangan dibuat di luar daripada keputusan mesyuarat, ini bukan keputusan parti.',
 '"Apa locus standy yang ada pada Setiausaha Kerja untuk 

In [20]:
t5.greedy_decoder(splitted)

['PELETAKAN jawatan Tun Dr Mahathir Mohamad sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) ditolak pada mesyuarat khas Majlis Pimpinan Tertinggi (MPT) pada 24 Februari lalu.',
 'Justeru, tidak timbul soal peletakan jawatan itu sah atau tidak kerana ia sudah pun diputuskan pada peringkat parti yang dipersetujui semua termasuk Presiden, Tan Sri Muhyiddin Yassin.',
 'Bekas Setiausaha Agung Bersatu, Datuk Marzuki Yahya, berkata pada mesyuarat itu MPT sebulat suara menolak peletakan jawatan Dr Mahathir.',
 '"Jadi ini agak berlawanan dengan keputusan yang kita buat.',
 'Saya tidak faham bagaimana Jabatan Pendaftaran Pertubuhan Malaysia (JPPM) kata peletakan jawatan itu sah sedangkan kita sudah membuat keputusan di dalam mesyuarat, bukan seorang dua yang membuat keputusan.',
 '"Semua keputusan mesti dibuat melalui parti.',
 'Walau apa pun perbincangan dibuat di luar keputusan mesyuarat, ini bukan keputusan parti.',
 '"Apa locus standy yang ada pada Setiausaha Kerja untuk membawa p